In [2]:
# Challenge Day 5:

import pandas as pd
from collections import Counter
from typing import List, Union

"""
Puzzle 1: Count number of points with >= 2 path* overlap
*horiz and vert paths considered

Method:
1) lazy pandas dataframe style (x1, y1, x2, y2, horiz/vert type) 
2) add data corresponding to path type
3) create dotplot grid ***eventually numpy array of arrays***, use 0's instead of .
4) loop through horiz type, assign values by list comp replacement
5) loop through vert type, transpose dotplot to assign identically to horiz type, then re-transpose
6) after loop, count any occurrences >= 2
"""

def csv_to_coord_df(path: str) -> pd.DataFrame:
    return(pd.read_csv(path, sep=",|->", names=['X1','Y1','X2','Y2', 'Direction'], engine='python')) # txt import separated by two delims , and ->

def create_dotplot(x_size: int, y_size: int) -> List[str]:
    dotplot = [["."] * x_size for dot_cols in range(y_size)]
    return(dotplot)

def max_XY(in_df: pd.DataFrame) -> int:
    max_Xval = max(in_df[["X1", "X2"]].max(axis = 0))
    max_Yval = max(in_df[["Y1", "Y2"]].max(axis = 0))
    return(max_Xval, max_Yval)

def add_direction(in_df: pd.DataFrame) -> pd.DataFrame:
    for index, row in in_df.iterrows():
        if row['X1'] == row['X2']:
            in_df.loc[index, 'Direction'] = "vert"
        if row['Y1'] == row['Y2']:
            in_df.loc[index, 'Direction'] =  "horiz"
    in_df = in_df.dropna(subset = ['Direction'])
    return(in_df)

def plot_paths(in_df: pd.DataFrame, in_dotplot: List[Union[str, int]], direction: str) -> List[Union[str, int]]:
    if direction == "vert":
        in_dotplot = list(map(list, zip(*in_dotplot)))
        col = 'X1'
        axis_st = 'Y1'
        axis_end = 'Y2'
    else:
        col = 'Y1'
        axis_st = 'X1'
        axis_end = 'X2'
    sort_df = in_df.sort_values(by = col)
    filtered_df = sort_df[sort_df['Direction'] == direction]
    for index, row in filtered_df.iterrows():
        row_ind = row[col]
        st = row[axis_st]
        end = row[axis_end]
        if st < end:
            in_dotplot[row_ind][st:end+1] = [1 if x == "." else 2 for x in in_dotplot[row_ind][st:end+1]]
        elif st > end:
            in_dotplot[row_ind][end:st+1] = [1 if x == "." else 2 for x in in_dotplot[row_ind][end:st+1]]
        else:
            in_dotplot[row_ind][st:end+1] = [1 if x == "." else 2 for x in in_dotplot[row_ind][st:end+1]]
    if direction == "vert":
        in_dotplot = list(map(list, zip(*in_dotplot)))
    return(in_dotplot)

def count_twos(in_dotplot: List[Union[str, int]]) -> int:
    counts = 0
    for listElem in in_dotplot:
        counts += listElem.count(2)
    return(counts)

def main():
    # setup dotplot *** Change to numpy array of 0's
    coord_df = csv_to_coord_df("Day-5-input.txt") # 1)
    directional_df = add_direction(coord_df) # 2)
    X_max, Y_max = max_XY(directional_df) # 3)
    path_dotplot = create_dotplot(X_max+1, Y_max+1) # 3)

    # fill with horiz, vert data
    horiz_filled = plot_paths(directional_df, path_dotplot, "horiz") # 5)
    vert_filled = plot_paths(directional_df, horiz_filled, "vert") # 6)
    return(count_twos(vert_filled)) # 7)

main()

4728

In [20]:
# Challenge Day 5:

from time import perf_counter as pc

import operator
import pandas as pd
from collections import Counter
from typing import List, Union

"""
Puzzle 2: Count number of points with >= 2 path* overlap
*all paths considered

Method:
1) lazy pandas dataframe style (x1, y1, x2, y2, horiz/vert/diag type) 
2) add data corresponding to path type
3) create dotplot grid ***eventually numpy array of arrays***, use 0's instead of .
4) loop through diag type, assign values based on signage and dist of Ydist and Xdist
5) loop through horiz type, assign values by list comp replacement
6) loop through vert type, transpose dotplot to assign identically to horiz type, then re-transpose
7) after loop, count any occurrences >= 2
"""

def csv_to_coord_df(path: str) -> pd.DataFrame:
    return(pd.read_csv(path, sep=",|->", names=['X1','Y1','X2','Y2', 'Direction'], engine='python')) # txt import separated by two delims , and ->

def create_dotplot(x_size: int, y_size: int) -> List[str]:
    dotplot = [["."] * x_size for dot_cols in range(y_size)]
    return(dotplot)

def max_XY(in_df: pd.DataFrame) -> int:
    max_Xval = max(in_df[["X1", "X2"]].max(axis = 0))
    max_Yval = max(in_df[["Y1", "Y2"]].max(axis = 0))
    return(max_Xval, max_Yval)

def add_direction(in_df: pd.DataFrame) -> pd.DataFrame:
    for index, row in in_df.iterrows():
        if row['X1'] == row['X2']:
            in_df.loc[index, 'Direction'] = "vert"
        elif row['Y1'] == row['Y2']:
            in_df.loc[index, 'Direction'] =  "horiz"
        else:
            in_df.loc[index, 'Direction'] =  "diag"
    in_df = in_df.dropna(subset = ['Direction'])
    return(in_df)

ops = {
    '+' : operator.add,
    '-' : operator.sub
    }

def eval_op(op1: int, oper: str, op2: int) -> int:
    op1, op2 = int(op1), int(op2)
    return ops[oper](op1, op2)

def set_ops(Xdist: int, Ydist: int) -> int:
    setX = '+'
    setY = '+'
    if Ydist < 0:
        setY = '-'
    if Xdist < 0:
        setX = '-'
    return(setX, setY)

def plot_diag(in_dotplot: List[Union[str, int]], x1: int, x2: int, y1: int, y2: int):
    Xdist = x2-x1
    Ydist = y2-y1
    X_op, Y_op = set_ops(Xdist, Ydist)
    tracker = 0
    while tracker <= abs(Xdist): # keep going til tracker covers dist
        Y_ind = eval_op(y1, Y_op, tracker)
        X_ind = eval_op(x1, X_op, tracker)
        if in_dotplot[Y_ind][X_ind] == ".": # check loc...if no prior val
            in_dotplot[Y_ind][X_ind] = 1 # "." -> 1
        else: # if already a 1
            in_dotplot[Y_ind][X_ind] = 2 # 1 -> 2
        tracker += 1

def plot_allDiags(in_df: pd.DataFrame, in_dotplot: List[Union[str, int]]) -> List[Union[str, int]]:
    """
    if diag, then increase vals by 1 along slope
    """
    coord_df_diags = in_df.sort_values(by = "X1")
    filtered_diags = coord_df_diags[coord_df_diags['Direction'] == "diag"]
    for index, row in filtered_diags.iterrows():
        plot_diag(in_dotplot, row["X1"], row["X2"], row["Y1"], row["Y2"])
    return(in_dotplot)

def plot_altpaths(in_df: pd.DataFrame, in_dotplot: List[Union[str, int]], direction: str) -> List[Union[str, int]]:
    if direction == "vert":
        in_dotplot = list(map(list, zip(*in_dotplot))) # transpose for row manipulation
        col = 'X1'
        axis_st = 'Y1'
        axis_end = 'Y2'
    else:
        col = 'Y1'
        axis_st = 'X1'
        axis_end = 'X2'
    sort_df = in_df.sort_values(by = col)
    filtered_df = sort_df[sort_df['Direction'] == direction]
    for index, row in filtered_df.iterrows():
        row_ind = row[col]
        st = row[axis_st]
        end = row[axis_end]
        if st < end:
            in_dotplot[row_ind][st:end+1] = [1 if x == "." else 2 for x in in_dotplot[row_ind][st:end+1]]
        elif st > end:
            in_dotplot[row_ind][end:st+1] = [1 if x == "." else 2 for x in in_dotplot[row_ind][end:st+1]]
        else:
            in_dotplot[row_ind][st:end+1] = [1 if x == "." else 2 for x in in_dotplot[row_ind][st:end+1]]
    if direction == "vert":
        in_dotplot = list(map(list, zip(*in_dotplot))) # transpose back to original state
    return(in_dotplot)

def count_twos(in_dotplot: List[Union[str, int]]) -> int:
    counts = 0
    for listElem in in_dotplot:
        counts += listElem.count(2)
    return(counts)

def main():
    # setup dotplot *** Change to numpy array of 0's
    coord_df = csv_to_coord_df("Day-5-input.txt") # 1)
    directional_df = add_direction(coord_df) # 2)
    X_max, Y_max = max_XY(directional_df) # 3)
    path_dotplot = create_dotplot(X_max+1, Y_max+1) # 3)

    # fill with diag, horiz, vert data
    diag_filled = plot_allDiags(directional_df, path_dotplot) # 4)
    horiz_filled = plot_altpaths(directional_df, diag_filled, "horiz") # 5)
    vert_filled = plot_altpaths(directional_df, horiz_filled, "vert") # 6)
    return(count_twos(vert_filled)) # 7)

t0 = pc() # time zero
main()
print('Time elapsed(main()): ', pc() - t0) # end time - time zero

Time elapsed(main()):  0.18563320799989924


# Version 2, using np arrays

In [14]:
# Challenge Day 5:

import numpy as np
import pandas as pd

"""
Puzzle 1: Count number of points with >= 2 path* overlap
*horiz and vert paths considered

Method:
1) lazy pandas dataframe style (x1, y1, x2, y2, horiz/vert type) 
2) add data corresponding to path type
3) create dotplot grid WITH numpy array of arrays, using 0's to rep for .
4) loop through horiz type, += 1 at locs by list comp across rows
5) loop through vert type, transpose dotplot, += 1 at locs by list comp across rows, then re-transpose
6) after loop, count any occurrences >= 2
"""

def csv_to_coord_df(path: str) -> pd.DataFrame:
    return(pd.read_csv(path, sep=",|->", names=['X1','Y1','X2','Y2', 'Direction'], engine='python')) # txt import separated by two delims , and ->

def create_np_dotplot(x_size: int, y_size: int) -> np.ndarray:
    np_dotplot = np.zeros((y_size, x_size), dtype = int)
    return(np_dotplot)

def max_XY(in_df: pd.DataFrame) -> int:
    max_Xval = max(in_df[["X1", "X2"]].max(axis = 0))
    max_Yval = max(in_df[["Y1", "Y2"]].max(axis = 0))
    return(max_Xval, max_Yval)

def add_direction(in_df: pd.DataFrame) -> pd.DataFrame:
    for index, row in in_df.iterrows():
        if row['X1'] == row['X2']:
            in_df.loc[index, 'Direction'] = "vert"
        if row['Y1'] == row['Y2']:
            in_df.loc[index, 'Direction'] =  "horiz"
    in_df = in_df.dropna(subset = ['Direction'])
    return(in_df)

def plot_paths(in_df: pd.DataFrame, in_dotplot: np.ndarray, direction: str) -> np.ndarray:
    if direction == "vert":
        in_dotplot = np.transpose(in_dotplot)
        col = 'X1'
        axis_st = 'Y1'
        axis_end = 'Y2'
    else:
        col = 'Y1'
        axis_st = 'X1'
        axis_end = 'X2'
    sort_df = in_df.sort_values(by = col)
    filtered_df = sort_df[sort_df['Direction'] == direction]
    for index, row in filtered_df.iterrows():
        row_ind = row[col]
        st = row[axis_st]
        end = row[axis_end]
        if st < end:
            in_dotplot[row_ind, st:end+1] += 1
        elif st > end:
            in_dotplot[row_ind, end:st+1] += 1
        else:
            in_dotplot[row_ind, st:end+1] += 1
    if direction == "vert":
        in_dotplot = np.transpose(in_dotplot)
    return(in_dotplot)

def count_stacked(in_dotplot: np.ndarray) -> int:
    stacked_paths = (in_dotplot >= 2).sum()
    return(stacked_paths)

def main():
    # setup dotplot numpy array of 0's
    coord_df = csv_to_coord_df("Day-5-input.txt") # 1)
    directional_df = add_direction(coord_df) # 2)
    X_max, Y_max = max_XY(directional_df) # 3)
    path_dotplot = create_np_dotplot(X_max+1, Y_max+1) # 3)

    # fill with horiz, vert data
    horiz_filled = plot_paths(directional_df, path_dotplot, "horiz") # 5)
    vert_filled = plot_paths(directional_df, horiz_filled, "vert") # 6)
    return(count_stacked(vert_filled)) # 7)

main()

4728

In [64]:
# Challenge Day 5:

from time import perf_counter as pc

import numpy as np
import operator
import pandas as pd

"""
Puzzle 2: Count number of points with >= 2 path* overlap
*all paths considered

Method:
1) lazy pandas dataframe style (x1, y1, x2, y2, horiz/vert/diag type) 
2) add data corresponding to path type
3) create dotplot grid WITH numpy array of arrays, using 0's to rep for .
4) loop through diag type, += 1 at locs based on signage and dist of Ydist and Xdist
5) loop through horiz type, += 1 at locs by list comp across rows
6) loop through vert type, transpose dotplot, += 1 at locs by list comp across rows, then re-transpose
7) after loop, count any occurrences >= 2
"""

def csv_to_coord_df(path: str) -> pd.DataFrame:
    return(pd.read_csv(path, sep=",|->", names=['X1','Y1','X2','Y2', 'Direction'], engine='python')) # txt import separated by two delims , and ->

def create_np_dotplot(x_size: int, y_size: int) -> np.ndarray:
    np_dotplot = np.zeros((y_size, x_size), dtype = int)
    return(np_dotplot)

def max_XY(in_df: pd.DataFrame) -> int:
    max_Xval = max(in_df[["X1", "X2"]].max(axis = 0))
    max_Yval = max(in_df[["Y1", "Y2"]].max(axis = 0))
    return(max_Xval, max_Yval)

def add_direction(in_df: pd.DataFrame) -> pd.DataFrame:
    for index, row in in_df.iterrows():
        if row['X1'] == row['X2']:
            in_df.loc[index, 'Direction'] = "vert"
        elif row['Y1'] == row['Y2']:
            in_df.loc[index, 'Direction'] =  "horiz"
        else:
            in_df.loc[index, 'Direction'] =  "diag"
    in_df = in_df.dropna(subset = ['Direction'])
    return(in_df)

ops = {
    '+' : operator.add,
    '-' : operator.sub
    }

def eval_op(op1: int, oper: str, op2: int) -> int:
    op1, op2 = int(op1), int(op2)
    return ops[oper](op1, op2)

def set_ops(Xdist: int, Ydist: int) -> int:
    setX = '+'
    setY = '+'
    if Ydist < 0:
        setY = '-'
    if Xdist < 0:
        setX = '-'
    return(setX, setY)

def plot_diag(in_dotplot: np.ndarray, x1: int, x2: int, y1: int, y2: int):
    Xdist = x2-x1
    Ydist = y2-y1
    X_op, Y_op = set_ops(Xdist, Ydist)
    tracker = 0
    while tracker <= abs(Xdist): # keep going til tracker covers dist
        Y_ind = eval_op(y1, Y_op, tracker)
        X_ind = eval_op(x1, X_op, tracker)
        in_dotplot[Y_ind, X_ind] += 1
        tracker += 1

def plot_allDiags(in_df: pd.DataFrame, in_dotplot: np.ndarray) -> np.ndarray:
    coord_df_diags = in_df.sort_values(by = "X1")
    filtered_diags = coord_df_diags[coord_df_diags['Direction'] == "diag"]
    for index, row in filtered_diags.iterrows():
        plot_diag(in_dotplot, row["X1"], row["X2"], row["Y1"], row["Y2"])
    return(in_dotplot)

def plot_altpaths(in_df: pd.DataFrame, in_dotplot: np.ndarray, direction: str) -> np.ndarray:
    if direction == "vert":
        in_dotplot = np.transpose(in_dotplot)
        col = 'X1'
        axis_st = 'Y1'
        axis_end = 'Y2'
    else:
        col = 'Y1'
        axis_st = 'X1'
        axis_end = 'X2'
    sort_df = in_df.sort_values(by = col)
    filtered_df = sort_df[sort_df['Direction'] == direction]
    for index, row in filtered_df.iterrows():
        row_ind = row[col]
        st = row[axis_st]
        end = row[axis_end]
        if st < end:
            in_dotplot[row_ind, st:end+1] += 1
        elif st > end:
            in_dotplot[row_ind, end:st+1] += 1
        else:
            in_dotplot[row_ind, st:end+1] += 1
    if direction == "vert":
        in_dotplot = np.transpose(in_dotplot)
    return(in_dotplot)

def count_stacked(in_dotplot: np.ndarray) -> int:
    stacked_paths = (in_dotplot >= 2).sum()
    return(stacked_paths)

def main():
    # setup dotplot numpy array of 0's
    coord_df = csv_to_coord_df("Day-5-input.txt") # 1)
    directional_df = add_direction(coord_df) # 2)
    X_max, Y_max = max_XY(directional_df) # 3)
    path_dotplot = create_np_dotplot(X_max+1, Y_max+1) # 3)

    # fill with diag, horiz, vert data
    diag_filled = plot_allDiags(directional_df, path_dotplot) # 4)
    horiz_filled = plot_altpaths(directional_df, diag_filled, "horiz") # 5)
    vert_filled = plot_altpaths(directional_df, horiz_filled, "vert") # 6)
    return(count_stacked(vert_filled)) # 7)

t0 = pc() # time zero
main()
print('Time elapsed(main()): ', pc() - t0) # end time - time zero

Time elapsed(main()):  0.15335029200014105


# Excessive timeit results

In [49]:
#Speedtest
import timeit

TIMES = 10 #num of executions

SETUP = """
import operator
import pandas as pd
from collections import Counter
from typing import List, Union

def csv_to_coord_df(path: str) -> pd.DataFrame:
    return(pd.read_csv(path, sep=",|->", names=['X1','Y1','X2','Y2', 'Direction'], engine='python')) # txt import separated by two delims , and ->

def create_dotplot(x_size: int, y_size: int) -> List[str]:
    dotplot = [["."] * x_size for dot_cols in range(y_size)]
    return(dotplot)

def max_XY(in_df: pd.DataFrame) -> int:
    max_Xval = max(in_df[["X1", "X2"]].max(axis = 0))
    max_Yval = max(in_df[["Y1", "Y2"]].max(axis = 0))
    return(max_Xval, max_Yval)

def add_direction(in_df: pd.DataFrame) -> pd.DataFrame:
    for index, row in in_df.iterrows():
        if row['X1'] == row['X2']:
            in_df.loc[index, 'Direction'] = "vert"
        elif row['Y1'] == row['Y2']:
            in_df.loc[index, 'Direction'] =  "horiz"
        else:
            in_df.loc[index, 'Direction'] =  "diag"
    in_df = in_df.dropna(subset = ['Direction'])
    return(in_df)

ops = {
    '+' : operator.add,
    '-' : operator.sub
    }

def eval_op(op1: int, oper: str, op2: int) -> int:
    op1, op2 = int(op1), int(op2)
    return ops[oper](op1, op2)

def set_ops(Xdist: int, Ydist: int) -> int:
    setX = '+'
    setY = '+'
    if Ydist < 0:
        setY = '-'
    if Xdist < 0:
        setX = '-'
    return(setX, setY)

def plot_diag(in_dotplot: List[Union[str, int]], x1: int, x2: int, y1: int, y2: int):
    Xdist = x2-x1
    Ydist = y2-y1
    X_op, Y_op = set_ops(Xdist, Ydist)
    tracker = 0
    while tracker <= abs(Xdist): # keep going til tracker covers dist
        Y_ind = eval_op(y1, Y_op, tracker)
        X_ind = eval_op(x1, X_op, tracker)
        if in_dotplot[Y_ind][X_ind] == ".": # check loc...if no prior val
            in_dotplot[Y_ind][X_ind] = 1 # "." -> 1
        else: # if already a 1
            in_dotplot[Y_ind][X_ind] = 2 # 1 -> 2
        tracker += 1

def plot_allDiags(in_df: pd.DataFrame, in_dotplot: List[Union[str, int]]) -> List[Union[str, int]]:
    coord_df_diags = in_df.sort_values(by = "X1")
    filtered_diags = coord_df_diags[coord_df_diags['Direction'] == "diag"]
    for index, row in filtered_diags.iterrows():
        plot_diag(in_dotplot, row["X1"], row["X2"], row["Y1"], row["Y2"])
    return(in_dotplot)

def plot_altpaths(in_df: pd.DataFrame, in_dotplot: List[Union[str, int]], direction: str) -> List[Union[str, int]]:
    if direction == "vert":
        in_dotplot = list(map(list, zip(*in_dotplot)))
        col = 'X1'
        axis_st = 'Y1'
        axis_end = 'Y2'
    else:
        col = 'Y1'
        axis_st = 'X1'
        axis_end = 'X2'
    sort_df = in_df.sort_values(by = col)
    filtered_df = sort_df[sort_df['Direction'] == direction]
    for index, row in filtered_df.iterrows():
        row_ind = row[col]
        st = row[axis_st]
        end = row[axis_end]
        if st < end:
            in_dotplot[row_ind][st:end+1] = [1 if x == "." else 2 for x in in_dotplot[row_ind][st:end+1]]
        elif st > end:
            in_dotplot[row_ind][end:st+1] = [1 if x == "." else 2 for x in in_dotplot[row_ind][end:st+1]]
        else:
            in_dotplot[row_ind][st:end+1] = [1 if x == "." else 2 for x in in_dotplot[row_ind][st:end+1]]
    if direction == "vert":
        in_dotplot = list(map(list, zip(*in_dotplot)))
    return(in_dotplot)

def count_twos(in_dotplot: List[Union[str, int]]) -> int:
    counts = 0
    for listElem in in_dotplot:
        counts += listElem.count(2)
    return(counts)

def main():
    # setup dotplot *** Change to numpy array of 0's
    coord_df = csv_to_coord_df("Day-5-input.txt") # 1)
    directional_df = add_direction(coord_df) # 2)
    X_max, Y_max = max_XY(directional_df) # 3)
    path_dotplot = create_dotplot(X_max+1, Y_max+1) # 3)

    # fill with diag, horiz, vert data
    diag_filled = plot_allDiags(directional_df, path_dotplot) # 4)
    horiz_filled = plot_altpaths(directional_df, diag_filled, "horiz") # 5)
    vert_filled = plot_altpaths(directional_df, horiz_filled, "vert") # 6)
    return(count_twos(vert_filled)) # 7)
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES) #store time taken for 10000 executions, repeated 5 times by default
    print(label, *('{:.3f}'.format(x) for x in res)) #round to 3 places after the decimal pt in Floating point decimal format

clock('no numpy       :', 'main()')

no numpy       : 1.743 1.715 1.670 1.637 1.637


In [53]:
#Speedtest
import timeit

TIMES = 10 #num of executions

SETUP = """
import numpy as np
import operator
import pandas as pd
from collections import Counter

def csv_to_coord_df(path: str) -> pd.DataFrame:
    return(pd.read_csv(path, sep=",|->", names=['X1','Y1','X2','Y2', 'Direction'], engine='python')) # txt import separated by two delims , and ->

def create_np_dotplot(x_size: int, y_size: int) -> np.ndarray:
    np_dotplot = np.zeros((y_size, x_size), dtype = int)
    return(np_dotplot)

def max_XY(in_df: pd.DataFrame) -> int:
    max_Xval = max(in_df[["X1", "X2"]].max(axis = 0))
    max_Yval = max(in_df[["Y1", "Y2"]].max(axis = 0))
    return(max_Xval, max_Yval)

def add_direction(in_df: pd.DataFrame) -> pd.DataFrame:
    for index, row in in_df.iterrows():
        if row['X1'] == row['X2']:
            in_df.loc[index, 'Direction'] = "vert"
        elif row['Y1'] == row['Y2']:
            in_df.loc[index, 'Direction'] =  "horiz"
        else:
            in_df.loc[index, 'Direction'] =  "diag"
    in_df = in_df.dropna(subset = ['Direction'])
    return(in_df)

ops = {
    '+' : operator.add,
    '-' : operator.sub
    }

def eval_op(op1: int, oper: str, op2: int) -> int:
    op1, op2 = int(op1), int(op2)
    return ops[oper](op1, op2)

def set_ops(Xdist: int, Ydist: int) -> int:
    setX = '+'
    setY = '+'
    if Ydist < 0:
        setY = '-'
    if Xdist < 0:
        setX = '-'
    return(setX, setY)

def plot_diag(in_dotplot: np.ndarray, x1: int, x2: int, y1: int, y2: int):
    Xdist = x2-x1
    Ydist = y2-y1
    X_op, Y_op = set_ops(Xdist, Ydist)
    tracker = 0
    while tracker <= abs(Xdist): # keep going til tracker covers dist
        Y_ind = eval_op(y1, Y_op, tracker)
        X_ind = eval_op(x1, X_op, tracker)
        in_dotplot[Y_ind, X_ind] += 1
        tracker += 1

def plot_allDiags(in_df: pd.DataFrame, in_dotplot: np.ndarray) -> np.ndarray:
    coord_df_diags = in_df.sort_values(by = "X1")
    filtered_diags = coord_df_diags[coord_df_diags['Direction'] == "diag"]
    for index, row in filtered_diags.iterrows():
        plot_diag(in_dotplot, row["X1"], row["X2"], row["Y1"], row["Y2"])
    return(in_dotplot)

def plot_altpaths(in_df: pd.DataFrame, in_dotplot: np.ndarray, direction: str) -> np.ndarray:
    if direction == "vert":
        in_dotplot = np.transpose(in_dotplot)
        col = 'X1'
        axis_st = 'Y1'
        axis_end = 'Y2'
    else:
        col = 'Y1'
        axis_st = 'X1'
        axis_end = 'X2'
    sort_df = in_df.sort_values(by = col)
    filtered_df = sort_df[sort_df['Direction'] == direction]
    for index, row in filtered_df.iterrows():
        row_ind = row[col]
        st = row[axis_st]
        end = row[axis_end]
        if st < end:
            in_dotplot[row_ind, st:end+1] += 1
        elif st > end:
            in_dotplot[row_ind, end:st+1] += 1
        else:
            in_dotplot[row_ind, st:end+1] += 1
    if direction == "vert":
        in_dotplot = np.transpose(in_dotplot)
    return(in_dotplot)

def count_stacked(in_dotplot: np.ndarray) -> int:
    stacked_paths = (in_dotplot >= 2).sum()
    return(stacked_paths)

def main():
    # setup dotplot numpy array of 0's
    coord_df = csv_to_coord_df("Day-5-input.txt") # 1)
    directional_df = add_direction(coord_df) # 2)
    X_max, Y_max = max_XY(directional_df) # 3)
    path_dotplot = create_np_dotplot(X_max+1, Y_max+1) # 3)

    # fill with diag, horiz, vert data
    diag_filled = plot_allDiags(directional_df, path_dotplot) # 4)
    horiz_filled = plot_altpaths(directional_df, diag_filled, "horiz") # 5)
    vert_filled = plot_altpaths(directional_df, horiz_filled, "vert") # 6)
    return(count_stacked(vert_filled)) # 7)
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES) #store time taken for 10000 executions, repeated 5 times by default
    print(label, *('{:.3f}'.format(x) for x in res)) #round to 3 places after the decimal pt in Floating point decimal format

clock('numpy arrays        :', 'main()')

numpy arrays        : 1.436 1.431 1.404 1.410 1.409
